In [ ]:
import json
import os
import time
from openai import OpenAI
from google.colab import userdata
import base64
import re
from google.colab import userdata
from google.colab import drive
from tqdm import tqdm

drive.mount('/content/drive')


def is_image_file(filename):
    return filename.lower().endswith(('.png', '.jpg', '.jpeg'))


def run_gpt4o_on_existing_json(image_root, input_json_path, model_name='gpt4o'):
    """Appends GPT-4o result to existing JSON file."""

    # Initialize OpenAI client
    api_key = userdata.get('openai')
    client = OpenAI(api_key=api_key)

    def encode_image(image_path):
        """Encode image to base64"""
        with open(image_path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')

    def process_image_with_gpt(image_path, prompt):
        """Process image using GPT-4o"""
        base64_image = encode_image(image_path)

        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": prompt},
                        {
                            "type": "image_url",
                            "image_url": {
                                "url": f"data:image/jpeg;base64,{base64_image}"
                            }
                        }
                    ]
                }
            ]
        )
        response_text = response.choices[0].message.content
        return response_text

    print("Initializing GPT-4o processing...")

    #Load current JSON
    with open(input_json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    start_time = time.time()

    prompt = (
        "Extract the Turkish text from the image exactly as it appears. "
        "Do not repeat, comment, translate, or add any text. "
        "Return only the raw text. If no text is detected, return an empty response."
    )

    for filename in tqdm([k for k in data if not k.startswith("_")], desc="OCR process"):
        # Yol oluştur
        for root, _, files in os.walk(image_root):
            if 'text_category' in root:
                continue
            if filename in files:
                img_path = os.path.join(root, filename)
                try:
                    extracted_text = process_image_with_gpt(img_path, prompt)

                    #write OCR results
                    data[filename]["models"][model_name] = {
                        "prediction": extracted_text,
                        "cer": None,
                        "wer": None
                    }

                except Exception as e:
                    print(f"{filename} error: {str(e)}")
                break  
    elapsed = round(time.time() - start_time,2)
    print(f"\n OCR completed: {elapsed:.2f} seconds")

    meta = data.get("_meta", {})
    processing_times = meta.get("processing_times", {})
    processing_times[model_name] = elapsed
    meta["processing_times"] = processing_times
    data["_meta"] = meta

    # save updated JSON
    with open(input_json_path, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

    print(f" Updated JSON saved: {input_json_path}")
    return data





input_json_path = '/content/drive/MyDrive/nutuk/benchmark/converted_data.json'
image_root = '/content/drive/MyDrive/nutuk/benchmark/'


# start OCR process
updated = run_gpt4o_on_existing_json(image_root, input_json_path)